In [6]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm
import glob
from sklearn.model_selection import train_test_split
from PIL import Image, ImageEnhance

# CONFIG
output_dir = "synthetic_squares"
img_dir = os.path.join(output_dir, "images")
lbl_dir = os.path.join(output_dir, "labels")
os.makedirs(img_dir, exist_ok=True)
os.makedirs(lbl_dir, exist_ok=True)

img_size = 224
total_images = 2000           # Total images (across train/val)
min_squares = 1
max_squares = 3               # Up to N colored squares per image
distractor_shapes = True      # Add circles/triangles for difficulty
blur_prob = 0.3
brightness_jitter = 0.25
real_bg_prob = 0.4            # Use real backgrounds X% of the time

colors = {
    "blue":   ((255,  30,  30), 0),
    "red":    ((30,  30, 255), 1),
    "orange": ((0, 165, 255), 2),
    "yellow": ((0, 255, 255), 3),
    "purple": ((200,  80, 200), 4),
    "brown":  ((19,  69, 139), 5)
}
class_names = list(colors.keys())

backgrounds = [
    (240,240,240), (255,255,255), (200,200,200), (160,160,160),
    (220,200,170), (230,230,210)
]

# Optional: supply your own real image backgrounds (landscape, desk, etc)
real_bg_folder = "real_backgrounds"
real_bg_files = glob.glob(os.path.join(real_bg_folder, "*.jpg"))

def make_background():
    if random.random() < real_bg_prob and real_bg_files:
        bg = Image.open(random.choice(real_bg_files)).resize((img_size, img_size)).convert("RGB")
        bg = np.array(bg)
    else:
        bg = np.full((img_size, img_size, 3), random.choice(backgrounds), dtype=np.uint8)
        # Add noise
        if random.random() < 0.4:
            noise = np.random.normal(0, 10, (img_size, img_size, 3))
            bg = np.clip(bg + noise, 0, 255).astype(np.uint8)
    return bg

def random_square_params():
    cx = random.randint(int(0.2*img_size), int(0.8*img_size))
    cy = random.randint(int(0.2*img_size), int(0.8*img_size))
    size = random.randint(int(0.18*img_size), int(0.5*img_size))
    
    # MODIFIED: Apply perspective transform with primary x-direction squeeze
    # and slight y-direction squeeze for more realistic perspective
    x_squeeze = random.uniform(0.4, 1.0)  # Primary squeeze in x direction (1.0 = no squeeze)
    y_squeeze = random.uniform(0.85, 1.0)  # Slight squeeze in y direction (1.0 = no squeeze)
    
    return cx, cy, size, x_squeeze, y_squeeze

def apply_random_blur(img):
    if random.random() < blur_prob:
        ksize = random.choice([3,5,7])
        img = cv2.GaussianBlur(img, (ksize,ksize), 0)
    if random.random() < 0.15:
        # Simulate motion blur
        size = random.choice([5,9,15])
        kernel_motion_blur = np.zeros((size, size))
        kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
        kernel_motion_blur = kernel_motion_blur / size
        img = cv2.filter2D(img, -1, kernel_motion_blur)
    return img

def apply_brightness(img):
    if random.random() < brightness_jitter:
        factor = random.uniform(0.7, 1.3)
        pil_img = Image.fromarray(img)
        enhancer = ImageEnhance.Brightness(pil_img)
        img = np.array(enhancer.enhance(factor))
    return img

def random_distractor(img):
    shape = random.choice(["circle","triangle"])
    color = tuple(np.random.randint(0,255,3).tolist())
    thickness = random.choice([-1,2])
    if shape == "circle":
        cx, cy = random.randint(0,img_size), random.randint(0,img_size)
        r = random.randint(15,50)
        cv2.circle(img, (cx,cy), r, color, thickness)
    elif shape == "triangle":
        pts = np.random.randint(0,img_size, (3,2))
        cv2.polylines(img, [pts], isClosed=True, color=color, thickness=thickness if thickness>0 else 2)
        if thickness == -1:
            cv2.fillPoly(img, [pts], color)
    return img

# --- Generation Loop ---
print("Generating images...")
all_image_paths = []
for i in tqdm(range(total_images)):
    img = make_background()
    label_lines = []
    nsq = random.randint(min_squares, max_squares)
    used_classes = random.sample(list(colors.keys()), nsq)
    for cname in used_classes:
        bgr, classid = colors[cname]
        cx, cy, size, x_squeeze, y_squeeze = random_square_params()
        half = size / 2
        
        # MODIFIED: Create square with primary x-dimension squeeze and slight y-dimension squeeze
        # This simulates more realistic perspective from floor where squares get thinner
        # mostly horizontally and slightly vertically
        pts = np.array([
            [-half * x_squeeze, -half * y_squeeze],  # Top-left - squeezed in x and slightly in y
            [half * x_squeeze, -half * y_squeeze],   # Top-right - squeezed in x and slightly in y
            [half, half],                           # Bottom-right - normal
            [-half, half]                           # Bottom-left - normal
        ], dtype=np.float32)
        
        # Apply random horizontal shear to simulate different viewing angles
        if random.random() < 0.7:
            shear_factor = random.uniform(-0.3, 0.3)
            shear_matrix = np.array([
                [1, shear_factor, 0],
                [0, 1, 0]
            ], dtype=np.float32)
            pts = np.dot(pts, shear_matrix[:,:2].T)
        
        pts += np.array([cx, cy])
        pts = pts.astype(np.int32)
        cv2.fillConvexPoly(img, pts, bgr)

        x_min, y_min = np.min(pts, axis=0)
        x_max, y_max = np.max(pts, axis=0)
        x_center = ((x_min + x_max) / 2) / img_size
        y_center = ((y_min + y_max) / 2) / img_size
        w = (x_max - x_min) / img_size
        h = (y_max - y_min) / img_size
        label_lines.append(f"{classid} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    # Add distractor shapes
    if distractor_shapes and random.random() < 0.5:
        n = random.randint(1,3)
        for _ in range(n):
            img = random_distractor(img)

    # Blurring & brightness
    img = apply_random_blur(img)
    img = apply_brightness(img)

    fname = f"synthetic_{i:05d}.jpg"
    img_path = os.path.join(img_dir, fname)
    lbl_path = os.path.join(lbl_dir, fname.replace(".jpg", ".txt"))
    cv2.imwrite(img_path, img)
    with open(lbl_path, "w") as f:
        for line in label_lines:
            f.write(line + "\n")
    all_image_paths.append(img_path)

print("Dataset generated. Splitting into train/val...")

# --- Train/Val Split ---
train_imgs, val_imgs = train_test_split(all_image_paths, test_size=0.15, random_state=42)
for split, img_list in [("train", train_imgs), ("val", val_imgs)]:
    split_img_dir = os.path.join(output_dir, "images", split)
    split_lbl_dir = os.path.join(output_dir, "labels", split)
    os.makedirs(split_img_dir, exist_ok=True)
    os.makedirs(split_lbl_dir, exist_ok=True)
    for img_path in img_list:
        base = os.path.basename(img_path)
        lbl_path = os.path.join(lbl_dir, base.replace(".jpg",".txt"))
        os.rename(img_path, os.path.join(split_img_dir, base))
        os.rename(lbl_path, os.path.join(split_lbl_dir, base.replace(".jpg",".txt")))

print("Done! Final structure is:")
print(f"{output_dir}/images/train/  (images for training)")
print(f"{output_dir}/images/val/    (images for validation)")
print(f"{output_dir}/labels/train/  (labels for training)")
print(f"{output_dir}/labels/val/    (labels for validation)")

# --- YAML for YOLOv8 ---
yaml = f"""
path: {os.path.abspath(output_dir)}
train: images/train
val: images/val
nc: {len(colors)}
names: {class_names}
"""
with open(os.path.join(output_dir, "data.yaml"), "w") as f:
    f.write(yaml)

print("data.yaml written!")

Generating images...


  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:07<00:00, 270.38it/s]


Dataset generated. Splitting into train/val...
Done! Final structure is:
synthetic_squares/images/train/  (images for training)
synthetic_squares/images/val/    (images for validation)
synthetic_squares/labels/train/  (labels for training)
synthetic_squares/labels/val/    (labels for validation)
data.yaml written!


In [6]:
# CONFIG
import  os
output_dir = "synthetic_squares"
img_dir = os.path.join(output_dir, "images")
lbl_dir = os.path.join(output_dir, "labels")


img_size = 224
total_images = 2000           # Total images (across train/val)
min_squares = 1
max_squares = 3               # Up to N colored squares per image
distractor_shapes = True      # Add circles/triangles for difficulty
blur_prob = 0.3
brightness_jitter = 0.25
real_bg_prob = 0.4            # Use real backgrounds X% of the time

In [8]:
# Training YOLOv8 on the synthetic dataset
import torch
from ultralytics import YOLO

# Configuration for YOLOv8 training
config = {
    "model_size": "n",        # n (nano), s (small), m (medium), l (large), x (xlarge)
    "epochs": 50,             # Number of training epochs
    "batch_size": 16,         # Batch size
    "img_size": img_size,     # Use same image size as generated dataset
    "patience": 10,           # Early stopping patience
    "device": 0 if torch.cuda.is_available() else "cpu"  # Use GPU if available
}

print(f"Training YOLOv8{config['model_size']} model on synthetic squares dataset")
print(f"Using device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

# Initialize pre-trained YOLOv8 model
model = YOLO(f"yolov8{config['model_size']}.pt")

# Train the model
results = model.train(
    data=os.path.join(output_dir, "data.yaml"),
    epochs=config['epochs'],
    imgsz=config['img_size'],
    batch=config['batch_size'],
    patience=config['patience'],
    device=config['device'],
    verbose=True
)

print("\nTraining complete!")
print(f"Model saved at: {os.path.join(model.trainer.save_dir, 'weights', 'best.pt')}")

# Validate the model
val_results = model.val()
print(f"Validation metrics: mAP@0.5={val_results.box.map50:.4f}, mAP@0.5:0.95={val_results.box.map:.4f}")

# Optional: Run inference on a sample image
sample_img = os.path.join(output_dir, "images", "val", os.listdir(os.path.join(output_dir, "images", "val"))[0])
results = model.predict(source=sample_img, save=True, conf=0.25)
print(f"Inference test complete. Results saved at: {model.predictor.save_dir}")

Training YOLOv8n model on synthetic squares dataset
Using device: CPU
Ultralytics 8.3.141  Python-3.9.18 torch-2.7.0+cpu CPU (AMD Ryzen 9 7950X 16-Core Processor)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=synthetic_squares\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, over

100%|██████████| 755k/755k [00:00<00:00, 9.05MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,012,018 parameters, 3,012,002 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access  (ping: 0.10.0 ms, read: 3.32.4 MB/s, size: 13.5 KB)


train: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares\labels\train... 1700 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1700/1700 [00:01<00:00, 1166.23it/s]


...

Optimizer stripped from runs\detect\train2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.2MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.141  Python-3.9.18 torch-2.7.0+cpu CPU (AMD Ryzen 9 7950X 16-Core Processor)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:01<00:00,  7.41it/s]


                   all        300        605      0.994      0.964      0.987      0.956
                  blue        106        106       0.99      0.955      0.984       0.95
                   red        105        105          1      0.988      0.995      0.974
                orange        105        105          1      0.957      0.994      0.971
                yellow         93         93      0.991          1      0.995      0.968
                purple        102        102      0.991      0.931      0.969      0.924
                 brown         94         94      0.989      0.954      0.985      0.949
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\train2

Training complete!
Model saved at: runs\detect\train2\weights\best.pt
Ultralytics 8.3.141  Python-3.9.18 torch-2.7.0+cpu CPU (AMD Ryzen 9 7950X 16-Core Processor)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image

val: Scanning C:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares\labels\val.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
c:\Users\fredr\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:01<00:00, 13.26it/s]


                   all        300        605      0.994      0.964      0.987      0.956
                  blue        106        106       0.99      0.955      0.984       0.95
                   red        105        105          1      0.988      0.995      0.974
                orange        105        105          1      0.957      0.994      0.971
                yellow         93         93      0.991          1      0.995      0.968
                purple        102        102      0.991      0.931      0.969      0.924
                 brown         94         94      0.989      0.954      0.985      0.949
Speed: 0.0ms preprocess, 3.0ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\train22
Validation metrics: mAP@0.5=0.9870, mAP@0.5:0.95=0.9560

image 1/1 c:\Users\fredr\BTH\Robotik Project\Robotics\21MajFiles\apa\synthetic_squares\images\val\synthetic_00023.jpg: 224x224 1 orange, 12.3ms
Speed: 0.5ms preprocess, 12.3ms inference, 1.0ms postproc

In [9]:
import os
from ultralytics import YOLO
import cv2

# === Paths ===
model_path = r"runs\detect\train2\weights\best.pt"
image_dir = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train"
output_dir = r"C:\Users\fredr\BTH\Robotik Project\Robotics\yolo_dataset\images\train_annotated"

# === Create output directory if it doesn't exist ===
os.makedirs(output_dir, exist_ok=True)

# === Load model ===
model = YOLO(model_path)

# === Inference on all images ===
image_extensions = ('.jpg', '.jpeg', '.png')
for filename in os.listdir(image_dir):
    if filename.lower().endswith(image_extensions):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)

        # Run inference
        results = model(image)[0]

        # Draw bounding boxes
        for box in results.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names[cls_id]

            # Get bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            color = (0, 255, 0)  # Green box
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            text = f"{label} {conf:.2f}"
            cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Save annotated image
        out_path = os.path.join(output_dir, filename)
        cv2.imwrite(out_path, image)

print(f"Done! Annotated images saved to: {output_dir}")



0: 224x224 (no detections), 8.5ms
Speed: 1.3ms preprocess, 8.5ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 8.3ms
Speed: 0.8ms preprocess, 8.3ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 1 brown, 8.8ms
Speed: 0.4ms preprocess, 8.8ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 8.8ms
Speed: 0.4ms preprocess, 8.8ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 8.7ms
Speed: 0.3ms preprocess, 8.7ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 8.3ms
Speed: 0.4ms preprocess, 8.3ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 8.0ms
Speed: 0.5ms preprocess, 8.0ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 8.4ms
Speed: 0.5ms preprocess, 8.4ms inference, 0.2ms postproc